In [1]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
def generate_basis(N_sites, N_particles): # Checked OK
    """Generate all Fock basis states with N_particles electrons."""
    basis = []
    for comb in combinations(range(N_sites), N_particles):
        state = [0] * N_sites
        for idx in comb:
            state[idx] = 1
        basis.append(tuple(state))
    return basis

In [ ]:
def fock_state_to_index(basis): # Checked OK
    """Map each Fock state to its index in the basis."""
    
    return {state: i for i, state in enumerate(basis)}

In [ ]:
def apply_annihilation(state, i): # Checked OK
    if state[i] == 0:
        return None, 0
    new_state = list(state)
    new_state[i] = 0
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

def apply_creation(state, i): # Checked OK
    if state[i] == 1:
        return None, 0
    new_state = list(state)
    new_state[i] = 1
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

In [21]:
def build_Hamiltonian(basis, N_unit_cells, v, w):
    dim = len(basis)
    H = csr_matrix((dim, dim), dtype=np.float64)
    state_to_idx = fock_state_to_index(basis)

    for idx, state in enumerate(basis):
        # Intra-cell hopping: A <-> B
        for i in range(N_unit_cells):
            site_A = 2*i
            site_B = 2*i + 1

            # B -> A
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

            # A -> B (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

        # Inter-cell hopping: A_{i+1} <-> B_i
        for i in range(N_unit_cells - 1):
            site_B = 2*i + 1
            site_A_next = 2*(i+1)

            # B_i -> A_{i+1}
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A_next)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

            # A_{i+1} -> B_i (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A_next)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

    return H

In [14]:
basis = generate_basis(4, 2)
print(basis)

[(1, 1, 0, 0), (1, 0, 1, 0), (1, 0, 0, 1), (0, 1, 1, 0), (0, 1, 0, 1), (0, 0, 1, 1)]


In [15]:
state_to_idx = fock_state_to_index(basis)
print(state_to_idx)

{(1, 1, 0, 0): 0, (1, 0, 1, 0): 1, (1, 0, 0, 1): 2, (0, 1, 1, 0): 3, (0, 1, 0, 1): 4, (0, 0, 1, 1): 5}


In [19]:
print(apply_creation(basis[0], 2))

((1, 1, 1, 0), 1)
